In [13]:
import pandas as pd
import requests
from collections import Counter
import sys

In [2]:
import argparse
# construct the argument parse and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-v", "--video",
                help="path to the (optional) video file")
args = vars(ap.parse_args())

usage: ipykernel_launcher.py [-h] [-v VIDEO]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\PHB\AppData\Roaming\jupyter\runtime\kernel-f0917f0f-66d4-48e6-bf3e-26a1af003f06.json


SystemExit: 2

D:\anaconda\envs\py37\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
chunksize = 100000
tfr = pd.read_csv('CONDITION_OCCURRENCE.tsv', chunksize=chunksize, iterator=True,sep='\t')
condition = pd.concat(tfr, ignore_index=True)

In [30]:
data = pd.read_csv('people_AD.tsv',sep='\t')
condition = pd.read_csv('Condition_Occurrence_Mapping_processed.csv')

In [31]:
def query(hpo_ids):
    query_result = []
    for hpo_id in hpo_ids:
        cor_gene = []
        hid = hpo_id.split('_')[1]
        response = requests.get(url = f"https://hpo.jax.org/api/hpo/term/HP%3A{hid}/genes?max=-1")
        #response.content
        try:
            response_info = response.json()['genes']
            for i in response_info:
                gene = i['geneSymbol']
                cor_gene.append(gene)
            query_result.append(cor_gene)
        except:
            print(sys.exc_info())
    return query_result

In [32]:
condition['ONTOLOGY_LOGIC'].unique()

array(['OR', nan, 'AND', 'INJURY', 'AND(0, 1, NOT(2))', 'NOT YET MAPPED',
       'FINDING', 'AND(0, NOT(1))', 'NOT'], dtype=object)

In [33]:
condition

,CONCEPT_ID,CONCEPT_LABEL,CONCEPT_TYPE,CONCEPT_NAME,CONCEPT_CODE,CONCEPT_VOCAB,CONCEPT_VOCAB_VERSION,CONCEPT_SYNONYM,UMLS_CUI,UMLS_SEMANTIC_TYPE,ANCESTOR_CONCEPT_ID,ANCESTOR_CONCEPT_CODE,ANCESTOR_CONCEPT_NAME,ANCESTOR_CONCEPT_VOCAB,ANCESTOR_CONCEPT_VOCAB_VERSION,ONTOLOGY_LOGIC,ONTOLOGY_URI,ONTOLOGY_LABEL,MAPPING_CATEGORY,MAPPING_EVIDENCE
0,81902,Urinary tract infectious disease,Concept Used In Practice,Urinary tract infectious disease,68566005.0,SNOMED,SnomedCT Release 20180131,Urinary tract infectious disease (disorder) | ...,C0042029,Disease or Syndrome,4028071 | 75865 | 81902 | 4274025 | 36102977 |...,404684003 | 10021902 | 10046694 | 10047461 | 1...,Renal and urinary disorders | Urogenital findi...,SNOMED | MedDRA,MedDRA version 19.1 | SnomedCT Release 20180131,OR,HP_0000119 | HP_0000010,abnormality of the genitourinary system | recu...,Automatic One-to-Many Ancestor,OBO_DbXref-OMOP_ANCESTOR_SOURCE_CODE:snomed_42...
1,134438,Contact dermatitis,Concept Used In Practice,Contact dermatitis,40275004.0,SNOMED,SnomedCT Release 20180131,Dermatitis venenata | CD - Contact dermatitis ...,C0011616,Disease or Syndrome,37320108 | 36000000 | 444112 | 37320089 | 4212...,10040836 | 19660004 | 10061366 | 10022891 | 10...,Clinical finding | Immune system disorders | G...,SNOMED | MedDRA,SnomedCT Release 20180131 | MedDRA version 19.1,NaN,HP_0032282,contact dermatitis,Automatic One-to-One Concept,OBO_LABEL-OMOP_ANCESTOR_LABEL:contact_dermatit...
2,192279,Disorder of kidney due to diabetes mellitus,Concept Used In Practice,Diabetic renal disease,127013003.0,SNOMED,SnomedCT Release 20180131,Diabetic renal disease (disorder) | Diabetic r...,C0011881,Disease or Syndrome,37103722 | 37019299 | 75865 | 4199402 | 363031...,10012655 | 118234003 | 10012653 | 734045002 | ...,Reproductive tract disorders NEC | Disease | K...,SNOMED | MedDRA,MedDRA version 19.1 | SnomedCT Release 20180131,AND,HP_0001259 | HP_0012211,coma | abnormal renal physiology,Manual One-to-Many Concept,Hand Mapping
3,195588,Cystitis,Concept Used In Practice,Cystitis,38822007.0,SNOMED,SnomedCT Release 20180131,Cystitis | Cystitis (disorder) | Bladder infec...,C0010692,Disease or Syndrome,36100000 | 36110716 | 43531061 | 37103722 | 36...,118234003 | 10046577 | 10005037 | 10046445 | 1...,Urogenital finding | Bladder and bladder neck ...,SNOMED | MedDRA,MedDRA version 19.1 | SnomedCT Release 20180131,NaN,HP_0100577,urinary bladder inflammation,Automatic One-to-One Concept,OBO_DbXref-OMOP_ANCESTOR_SOURCE_CODE:snomed_38...
4,321042,Cardiac arrest,Concept Used In Practice,Cardiac arrest,410429000.0,SNOMED,SnomedCT Release 20180131,Cardiac arrest (disorder) | Cardiac arrest,C0018790,Disease or Syndrome,440142 | 321042 | 37602356 | 37200000 | 352049...,cohort:500000901 | 404684003 | cohort:50000090...,"Cardiac disorders NEC | Investigations, imagin...",MedDRA | SNOMED | Cohort,SnomedCT Release 20180131 | MedDRA version 19....,NaN,HP_0001695,cardiac arrest,Automatic One-to-One Concept,OBO_DbXref-OMOP_CONCEPT_SOURCE_CODE:snomed_410...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,4172829,Limited social contact,Standard Concept Not Used In Practice,Limited social contact,423315002.0,SNOMED,SnomedCT Release 20180131,Limited social contact | Limited social contac...,C0237170,Finding,36903632 | 441840 | 36919117 | 36900000 | 3631...,250171008 | 404684003 | 106024000 | 10022891 |...,"Clinical finding | Investigations, imaging and...",SNOMED | MedDRA,MedDRA version 19.1 | SnomedCT Release 20180131,NaN,HP_0000708,behavioral abnormality,Automatic One-to-One Ancestor,OBO_hasExactSynonym-OMOP_ANCESTOR_LABEL:psychi...
206,40491961,Sepsis due to Pseudomonas,Concept Used In Practice,Sepsis due to Pseudomonas,448813005.0,SNOMED,SnomedCT Release 20180131,Sepsis due to Pseudomonas | Sepsis caused by P...,C2887096,Disease or Syndrome,36102144 | 37602357 | 37604017 | 435742 | 1327...,10051379 | 10021902 | 10040054 | 87628006 | 10...,Bacterial sepsis | Viral infection 

In [34]:
def count_score(gene_list,logic):
    final = []
    if logic == 'AND':
        for l in gene_list:
            final += l
    elif logic == 'OR':
        final = gene_list[0]
    elif logic ==' NOT':
        final = []
    else:
        final = gene_list[0]
        
    return final

In [35]:
def gene_dict(councept_count,data_filter):
        # version 1

    gene_dict = {}

    for n in range(concept_count.shape[0]):
        con_id = concept_count.index.values[n]
        n_con = concept_count.iloc[n,0]
        #print(con_id, ':', n_con)
        hpo_ids = data_filter.loc[data_filter['CONCEPT_ID'] == con_id, 'ONTOLOGY_URI'].to_string(index = False).split(' | ')
        #print(hpo_ids)
        map_cat = data_filter.loc[data_filter['CONCEPT_ID'] == con_id, 'MAPPING_CATEGORY'].to_string(index = False)
        logic = data_filter.loc[data_filter['CONCEPT_ID'] == con_id, 'ONTOLOGY_LOGIC'].to_string(index = False)
        #print(logic)
        if map_cat != 'Unmapped':
            query_res = query(hpo_ids)
            #print(len(query_res), map_cat)
            #print("query result: ", query_res)
            final = count_score(query_res,logic)
            #print("final:", final)
            if 'Concept' in map_cat:
                for gene in final:
                    if gene not in gene_dict:
                        gene_dict[gene] = 1 * n_con
                    else:
                        gene_dict[gene] += 1 * n_con
            elif 'Ancestor'in map_cat:
                for gene in final:
                    if gene not in gene_dict:
                        gene_dict[gene] = 0.5 * n_con
                    else:
                        gene_dict[gene] += 0.5 * n_con
        else:
            print('Unmapped concept ID:', con_id)

                #print(id_n)
        #print(hpo_id,'+',map_cat)
    return gene_dict

In [36]:
final_gene = Counter({})
for person in data['person_id'].unique():
    record = data.loc[data['person_id'] == person,:]
    concept_count = record.groupby('condition_concept_id').count().iloc[:,:1]
    res = Counter(gene_dict(concept_count,condition))
    final_gene = final_gene + res
    print(person)
    #print(res)

Unmapped concept ID: 4152936
7
(<class 'requests.exceptions.JSONDecodeError'>, JSONDecodeError('Expecting value: line 1 column 1 (char 0)'), <traceback object at 0x000001D0BA682B48>)
Unmapped concept ID: 4152936
Unmapped concept ID: 45768458
16
Unmapped concept ID: 78272
Unmapped concept ID: 4156265
Unmapped concept ID: 45768458
50
Unmapped concept ID: 45768458
83
Unmapped concept ID: 45768458
143
Unmapped concept ID: 4155034
Unmapped concept ID: 45768458
154
Unmapped concept ID: 45768458
255
Unmapped concept ID: 4155034
Unmapped concept ID: 45768458
266
Unmapped concept ID: 45768458
299
(<class 'requests.exceptions.JSONDecodeError'>, JSONDecodeError('Expecting value: line 1 column 1 (char 0)'), <traceback object at 0x000001D0B89E9FC8>)
(<class 'requests.exceptions.JSONDecodeError'>, JSONDecodeError('Expecting value: line 1 column 1 (char 0)'), <traceback object at 0x000001D0B89F7248>)
(<class 'requests.exceptions.JSONDecodeError'>, JSONDecodeError('Expecting value: line 1 column 1 (ch

2552
2561
(<class 'requests.exceptions.JSONDecodeError'>, JSONDecodeError('Expecting value: line 1 column 1 (char 0)'), <traceback object at 0x000001D0BA1ACD08>)
Unmapped concept ID: 4218389
Unmapped concept ID: 45768458
2605
(<class 'requests.exceptions.JSONDecodeError'>, JSONDecodeError('Expecting value: line 1 column 1 (char 0)'), <traceback object at 0x000001D0BA589588>)
2749
Unmapped concept ID: 4152936
Unmapped concept ID: 45768458
2785
Unmapped concept ID: 4156265
Unmapped concept ID: 45768458
2791
(<class 'requests.exceptions.JSONDecodeError'>, JSONDecodeError('Expecting value: line 1 column 1 (char 0)'), <traceback object at 0x000001D0B8A83BC8>)
(<class 'requests.exceptions.JSONDecodeError'>, JSONDecodeError('Expecting value: line 1 column 1 (char 0)'), <traceback object at 0x000001D0B8943AC8>)
Unmapped concept ID: 45768458
2799
(<class 'requests.exceptions.JSONDecodeError'>, JSONDecodeError('Expecting value: line 1 column 1 (char 0)'), <traceback object at 0x000001D0B8A1C208>

Unmapped concept ID: 45768458
5295
(<class 'requests.exceptions.JSONDecodeError'>, JSONDecodeError('Expecting value: line 1 column 1 (char 0)'), <traceback object at 0x000001D0BA717908>)
5300
(<class 'requests.exceptions.JSONDecodeError'>, JSONDecodeError('Expecting value: line 1 column 1 (char 0)'), <traceback object at 0x000001D0BA217448>)
Unmapped concept ID: 45768458
5313
Unmapped concept ID: 4146173
5393
5403
(<class 'requests.exceptions.JSONDecodeError'>, JSONDecodeError('Expecting value: line 1 column 1 (char 0)'), <traceback object at 0x000001D0BA26CCC8>)
Unmapped concept ID: 45768458
(<class 'requests.exceptions.JSONDecodeError'>, JSONDecodeError('Expecting value: line 1 column 1 (char 0)'), <traceback object at 0x000001D0B8B17088>)
5468
Unmapped concept ID: 45768458
5487
Unmapped concept ID: 81151
5535
Unmapped concept ID: 45768458
5563
Unmapped concept ID: 4156265
5585
(<class 'requests.exceptions.JSONDecodeError'>, JSONDecodeError('Expecting value: line 1 column 1 (char 0)'

Unmapped concept ID: 4155034
7470
7493
Unmapped concept ID: 4113008
Unmapped concept ID: 45768458
7579
(<class 'requests.exceptions.JSONDecodeError'>, JSONDecodeError('Expecting value: line 1 column 1 (char 0)'), <traceback object at 0x000001D0B89E4F88>)
Unmapped concept ID: 4146173
Unmapped concept ID: 45768458
7678
7857
Unmapped concept ID: 4218389
Unmapped concept ID: 37311061
Unmapped concept ID: 45768458
7904
Unmapped concept ID: 45768458
7921
(<class 'requests.exceptions.JSONDecodeError'>, JSONDecodeError('Expecting value: line 1 column 1 (char 0)'), <traceback object at 0x000001D0BA217E48>)
Unmapped concept ID: 45768458
7978
Unmapped concept ID: 4156265
Unmapped concept ID: 4218389
Unmapped concept ID: 45768458
8056
(<class 'requests.exceptions.JSONDecodeError'>, JSONDecodeError('Expecting value: line 1 column 1 (char 0)'), <traceback object at 0x000001D0B8B4E448>)
Unmapped concept ID: 4296204
Unmapped concept ID: 45768458
8192
8197
Unmapped concept ID: 78272
8344
(<class 'reque

KeyboardInterrupt: 

In [37]:
print(dict(final_gene))
gene_frame = pd.DataFrame(final_gene.items(), columns=['SYMBOL', 'Score'])

{'ABCC9': 628.0, 'ADAMTS2': 473.5, 'ADAMTSL2': 493.0, 'ADCY10': 86.5, 'AEBP1': 230.0, 'AIP': 4359.0, 'ALB': 701.5, 'ANAPC1': 1110.5, 'ANTXR2': 598.5, 'ARMC5': 4182.0, 'ASAH1': 4722.0, 'ASXL1': 4684.5, 'ASXL2': 119.5, 'ATP6V0A1': 4077.0, 'ATP7A': 1062.5, 'ATP7B': 4237.0, 'ATRX': 4934.0, 'B3GALT6': 788.0, 'B3GAT3': 110.5, 'BANF1': 190.5, 'BAZ1B': 1950.5, 'BCL7B': 1950.5, 'BMP1': 78, 'BMP15': 126.5, 'BMP2': 543.0, 'BMP6': 4030.5, 'BNC1': 126.5, 'BRAF': 4886.5, 'BUD23': 1950.5, 'CALCR': 69, 'CANT1': 94, 'CAVIN1': 847.0, 'CBL': 256.5, 'CBS': 652.0, 'CCN6': 100.5, 'CDC73': 514.5, 'CDH23': 4847.5, 'CHD7': 4346.0, 'CHST3': 119.0, 'CLIP2': 1950.5, 'COL1A1': 550.0, 'COL1A2': 512.0, 'COL2A1': 860.5, 'COL7A1': 699.5, 'CPLX1': 668.0, 'CTBP1': 668.0, 'CTC1': 774.0, 'CTDP1': 138.5, 'CYB5A': 136.5, 'CYP11A1': 122.5, 'CYP17A1': 182.0, 'CYP19A1': 238.0, 'CYP27A1': 4084.5, 'DHX37': 158.5, 'DKC1': 742.0, 'DKK1': 532.5, 'DMRT3': 142.5, 'DNAJC30': 1950.5, 'DPAGT1': 4205.0, 'DUSP6': 568.5, 'EIF2AK3': 783.5, 

In [39]:
gene_frame = gene_frame.sort_values(by=['Score'],ascending=False)
print(gene_frame['SYMBOL'])

405         FMR1
601        PSEN1
434         IFNG
581         TSC2
580         TSC1
          ...   
4512       KRT74
4513      HS3ST6
4514      SLC51A
4515    TMPRSS15
4511          C2
Name: SYMBOL, Length: 4516, dtype: object


In [43]:
gene_frame.iloc[0:200,].to_csv('genes_ad_hpo.csv',index = False)

In [28]:
gene_frame['Score'] /279

1       24.037634
25      23.919355
348     23.747312
314     23.431900
149     23.198925
          ...    
4639     0.001792
4638     0.001792
4637     0.001792
4636     0.001792
4665     0.001792
Name: Score, Length: 4729, dtype: float64

In [18]:
data['person_id'].unique()

array([     30,      40,     117, ..., 1141503, 1141571, 1141580],
      dtype=int64)

In [23]:
data['person_id'].unique().index(4408)

AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [41]:
print(person)

8835


In [42]:
np.where(data['person_id'].unique()==8835)

(array([246], dtype=int64),)

In [26]:
import numpy as np

In [29]:
gene_frame

,SYMBOL,Score
1,ABCC8,6706.5
25,CARS1,6673.5
348,IFNG,6625.5
314,FMR1,6537.5
149,PRKAR1A,6472.5
...,...,...
4639,MYO6,0.5
4638,MYH2,0.5
4637,MYF5,0.5
4636,MSRB3,0.5
